# Import Files

In [1]:
from os import listdir
from os.path import isfile, join
mypath = "../images"
file_names = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(str(len(file_names)) + ' images loaded')

97 images loaded


# Sift and Glcm 

In [2]:
import cv2
import numpy as np
from skimage.feature import  graycomatrix, graycoprops
images_sift = []
glcm=[]
labels = []
size = 128
sift = cv2.SIFT_create()
cataract=0
normal=0
for i, file in enumerate(file_names):
        image = cv2.imread(mypath+"/"+file,0)
        h,w=image.shape
#         if(h>128 and w>128):
        image = cv2.resize(image, (size, size), interpolation = cv2.INTER_AREA)
        img_arr = np.array(image)
        gCoMat = graycomatrix(img_arr, [1], [0],256,symmetric=True, normed=True) # Co-occurance matrix
        contrast = graycoprops(gCoMat, prop='contrast')[0][0]
        dissimilarity = graycoprops(gCoMat, prop='dissimilarity')[0][0]
        homogeneity = graycoprops(gCoMat, prop='homogeneity')[0][0]
        energy = graycoprops(gCoMat, prop='energy')[0][0]
        correlation = graycoprops(gCoMat, prop='correlation')[0][0]
        keypoints, descriptors = sift.detectAndCompute(image,None)
        descriptors=np.array(descriptors)
        descriptors=descriptors.flatten()
        glcm.append([contrast,dissimilarity,homogeneity,energy,correlation])
        images_sift.append(descriptors[:2304])

        #print(descriptors.shape)
        if file_names[i][0] == "c":
            cataract+=1
            labels.append(1)
        if file_names[i][0] == "n":
            normal+=1
            labels.append(0)

print("Testing and validation split done!")


Testing and validation split done!


In [3]:
labels

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [4]:
len(labels)

97

In [5]:
images_sift=np.array(images_sift)
images_sift.shape

(97, 2304)

In [6]:
glcm=np.array(glcm)
images_sift_glcm=np.concatenate((images_sift,glcm),axis=1)
images_sift_glcm.shape

(97, 2309)

In [7]:
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.metrics import confusion_matrix  
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# rf_class = RandomForestClassifier(n_estimators=100, criterion='entropy',max_features='sqrt', max_depth=3, n_jobs=2,
# random_state=1337)
# svm_poly=svm.SVC(kernel='poly',gamma=0.1,C=10)
# knn= KNeighborsClassifier(n_neighbors=6, metric='minkowski', p=2) 
log = LogisticRegression(solver='saga',penalty='l1')

# model_names={"Random Forest":rf_class,"SVM":svm_poly,"k nearest neighbor":knn,"logistic regression":log}

In [8]:
# def testing(model_name,X_train, X_test, y_train, y_test):
#     model=model_names[model_name]
#     model.fit(X_train,y_train)
#     yhat = model.predict(X_test)
#     # evaluate predictions
#     acc = accuracy_score(y_test, yhat)
#     print(model_name,'\tAccuracy: %.3f' % acc)
#     print(confusion_matrix(y_test, yhat))
#     print("\n\n")

In [9]:
# def result(dataset):
#     #Normalization
#     #min_max_scaler = preprocessing.StandardScaler()
#     #x_scaled = min_max_scaler.fit_transform(dataset)
    
    
#     #panda dataframe 
#     df=pd.DataFrame(data=dataset)
#     df['label']=labels
#     df=df.sample(frac=1)
#     X=df.drop(['label'], axis = 1)
#     y=df['label']
    
    
#     #Different model
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
#     for model in ["Random Forest","SVM","k nearest neighbor","logistic regression"]:
#         testing(model,X_train, X_test, y_train, y_test)

In [11]:
df=pd.DataFrame(data=images_sift_glcm)
df['label']=labels
df=df.sample(frac=1)
X=df.drop(['label'], axis = 1)
y=df['label']
#Different model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [13]:
log.fit(X_train,y_train)
yhat = log.predict(X_test)
# evaluate predictions
acc = accuracy_score(y_test, yhat)
print('\tAccuracy: %.3f' % acc)
print(confusion_matrix(y_test, yhat))
print("\n\n")

	Accuracy: 0.920
[[10  0]
 [ 2 13]]





c:\users\nishi.suraj-pc\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


# Saving model

In [14]:
from joblib import dump, load
dump(log, 'Phase1.joblib')

['Phase1.joblib']

In [31]:
model = load('Phase1.joblib') 
features = np.array([images_sift_glcm[67]])
print(model.predict(features))

[0]


# Test model

In [32]:
glcm_test, images_sift_test =[],[]
glcm_sift_test = []

In [40]:
image = cv2.imread('test1.jpg')
# h,w=image.shape
# if(h>128 and w>128):
image = cv2.resize(image, (size, size), interpolation = cv2.INTER_AREA)
img_arr = np.array(image)
gCoMat = graycomatrix(img_arr, [1], [0],256,symmetric=True, normed=True) # Co-occurance matrix
contrast = graycoprops(gCoMat, prop='contrast')[0][0]
dissimilarity = graycoprops(gCoMat, prop='dissimilarity')[0][0]
homogeneity = graycoprops(gCoMat, prop='homogeneity')[0][0]
energy = graycoprops(gCoMat, prop='energy')[0][0]
correlation = graycoprops(gCoMat, prop='correlation')[0][0]
keypoints, descriptors = sift.detectAndCompute(image,None)
descriptors=np.array(descriptors)
descriptors=descriptors.flatten()
glcm_test.append([contrast,dissimilarity,homogeneity,energy,correlation])
images_sift_test.append(descriptors[:2304])

ValueError: The parameter `image` must be a 2-dimensional array

In [37]:
from matplotlib import pyplot as plt
image = cv2.imread('test1.jpg')
# plt.imshow(image)
image

array([[[168, 192, 238],
        [165, 191, 237],
        [162, 188, 235],
        ...,
        [110, 134, 194],
        [109, 133, 193],
        [109, 133, 193]],

       [[169, 193, 239],
        [165, 191, 237],
        [161, 189, 236],
        ...,
        [110, 134, 194],
        [109, 133, 193],
        [109, 133, 193]],

       [[168, 194, 240],
        [166, 192, 238],
        [162, 190, 237],
        ...,
        [110, 134, 194],
        [109, 133, 193],
        [109, 133, 193]],

       ...,

       [[114, 150, 214],
        [114, 150, 214],
        [115, 151, 215],
        ...,
        [111, 148, 210],
        [110, 146, 206],
        [123, 157, 217]],

       [[113, 149, 213],
        [114, 150, 214],
        [115, 151, 215],
        ...,
        [129, 166, 228],
        [134, 171, 231],
        [123, 159, 219]],

       [[113, 149, 213],
        [114, 150, 214],
        [115, 151, 215],
        ...,
        [128, 165, 227],
        [133, 170, 232],
        [122, 158, 218]]

In [39]:
size

128